In [ ]:
import argparse
import datetime
import json
import math
import os
import random
import time
from pathlib import Path

import numpy as np
import ruamel.yaml as yaml
import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torchvision.utils as vutils
import torchvision.transforms as transforms
from PIL import Image

import utils
from dataset import create_dataset, create_sampler, create_loader
from dataset.utils import collect_tensor_result, grounding_eval_bbox, grounding_eval_bbox_vlue
from models.model_bbox import XVLM
from models.tokenization_bert import BertTokenizer
from models.tokenization_roberta import RobertaTokenizer
from optim import create_optimizer
from refTools.refer_python3 import REFER
from scheduler import create_scheduler
from utils.hdfs_io import hmkdir, hcopy, hexists

In [ ]:
torch.stack([torch.tensor([33., 14., 0., 1.]), torch.tensor([333., 514., 20., 105.])]).T

In [ ]:
def plot_bbox_prompts(image_int, coords, prompts):
    # Plot the resulting predictions on top of the original image
    plt.figure(figsize=(10,20))
    bbox_image = vutils.draw_bounding_boxes(image_int,coords, 
                                            width=3, 
                                            labels=[f"Prompt {i}" for i in range(len(prompts))],
                                            colors="red")

    plt.imshow(bbox_image.permute(1,2,0))
    plt.xlabel('\n' + '\n'.join([f'Prompt {i}: ' + prompts[i] for i in range(len(prompts))]), fontsize=14)
    plt.scatter(coords[0][0], coords[0][1], color='blue')
    plt.show()
    
plot_transform = transforms.Compose([
    transforms.Resize((config['image_res'], config['image_res']), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.uint8),
])

plot_transform2 = transforms.Compose([
#     transforms.Resize((config['image_res'], config['image_res']), interpolation=Image.BICUBIC),
#     transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float32),
    transforms.ConvertImageDtype(torch.uint8),
])

In [ ]:
config_file = "configs/Grounding_bbox_PascalParts.yaml"
config = yaml.load(open(config_file, 'r'), Loader=yaml.Loader)
# checkpoint = "model_checkpoints/16m_base_finetune/refcoco_bbox/checkpoint_best.pth"
evaluate = False
device = 6


print("Creating dataset")
grd_train_dataset, grd_test_dataset = create_dataset('pascalparts_grounding_bbox', config, evaluate=False)
# grd_train_dataset, grd_test_dataset = create_dataset('grounding_bbox', config, evaluate=False)

In [ ]:
grd_train_dataset.refer.refToAnn[0]['bbox']

In [ ]:
# grd_train_dataset.refer.Refs[0]
grd_train_dataset[0][2]

In [ ]:
# [grd_test_dataset[i] for i in range(10)]

In [ ]:
# img, text, bbox = grd_test_dataset[7]
# plt.imshow(img.permute(1,2,0))
# plt.xlabel(text)
# plt.show()


In [ ]:
i = 0
img, caption, bbox = grd_train_dataset[i]

# ann = grd_train_dataset.ann[i]
# true_bbox = torch.tensor(grd_train_dataset.refer.refToAnn[ann['ref_id']]['bbox'])

# bbox.shape
image_int = plot_transform(Image.open('./images/coco/train2014/COCO_train2014_000000581857.jpg'))

results = bbox.unsqueeze(0)*384
cx, cy, w, h = results[:,0], results[:,1], results[:,2], results[:,3]
coords = torch.stack((cx - w/2, cy-h/2, cx + w/2, cy + h/2)).T
# coords = torch.stack((cx,cy,cx+w, cy + h)).T

# print(cx,cy,w,h, cx+w, cy -h)






# plot_bbox_prompts(image_int, torch.tensor(true_bbox).unsqueeze(0), prompts=[caption])
# plot_bbox_prompts(image_int, coords, prompts=[caption])
# print(true_bbox)

# plt.imshow(img.permute(1,2,0))
# plt.show()





plot_bbox_prompts(plot_transform2(((img - img.min(1, keepdim=True)[0])/img.max(1, keepdim=True)[0]).clip(0,0.99)), coords, prompts=[caption])

In [ ]:
grd_train_dataset.img_ids

In [ ]:
grd_train_dataset.refer.refToAnn[ann['ref_id']]['bbox']

In [ ]:
# # [x['text'] for i,x in enumerate(grd_train_dataset)]
# for x in grd_train_dataset:
#     print(x)
# [x['text'] for x in grd_test_dataset.ann]
# grd_test_dataset.ann[:3]